# Importing the Dataset

In [2]:
#Importing the dataset
import pandas as pd 
df = pd.read_json("C:/Users/seerl/Downloads/Arts_Crafts_and_Sewing_5.json",lines=True)
df.sample(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
87641,5,True,"08 9, 2016",A2YCOTF2HNGDRH,B000WURGNC,{'Color:': ' Surf'},booger,"I haven't used this set for long, or even for ...",Do it,1470700800,NaN,NaN
448850,5,True,"02 28, 2018",A1D6Q96CAKN8X8,B0068RP090,NaN,SVH,looks wonderful in resin!,Five Stars,1519776000,NaN,NaN
248717,3,True,"09 12, 2014",A2RC9W12WGIG2P,B006U9CN8W,NaN,Cedric,Initial impression is that the motor is weak. ...,Available cheaper at Harbor Tools,1410480000,3,NaN
422181,5,True,"05 12, 2017",ASYTFHX745CLX,B000UXFIFO,NaN,DK,So I used the regular gingher scissors on all ...,Wow,1494547200,NaN,NaN
81873,5,True,"02 13, 2016",AVAUVU6DF0K0R,B00172XBQA,NaN,JAC,Very nice coloring! Looks great for hats and s...,Five Stars,1455321600,NaN,NaN
416304,5,True,"06 19, 2017",A3EY5CHO9Y7R6B,B01GGUAIX6,NaN,Chris,very sharp... cuts paper like butter.... I b...,very sharp... cuts paper like butter.... I b...,1497830400,NaN,NaN
479295,5,True,"05 14, 2015",A3HKJOK9FZIYKL,B00OI1MKSM,NaN,cmoge,Fun,Five Stars,1431561600,NaN,NaN
333070,5,True,"05 7, 2016",A2CQ99W0NFVGEE,B00JJ60BBE,{'Color:': ' #3023-light-gray'},Tab,Just what I wanted,Superior Threads,1462579200,NaN,NaN
333232,5,True,"03 13, 2015",AMK7D3YLOBQFV,B00JKR7OZS,NaN,MM3860,Great deal,Five Stars,1426204800,NaN,NaN
213693,5,True,"12 30, 2015",A2LASORRVJQI66,B004KQOLB4,NaN,C. Peters,Great quality wood. I am using them (I purcha...,Great quality wood... many opportunities for c...,1451433600,NaN,NaN


# Downloading the opinion_lexicon package

In [3]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
      opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\seerl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\opinion_lexicon.zip.


Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


# Creating Dictionary

In [4]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
 
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
        word_dict[word] = pos_score
      
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
        word_dict[word] = neg_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seerl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


# Bing liu Function

In [6]:
# Calculating the bing_liu score by adding the scores in dictionary
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score  


# Checking for empty spaces and appling the bing_liu function

In [8]:
# Checking for empty spaces and filling them with 'no review'
df['text'].fillna('no review', inplace=True)

# Apllying the bing_liu function
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)


# Displaying the text with bing liu scores

In [9]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


# Grouping the bing liu score by overall column

In [10]:
# In this step, we will take the mean of all the bing liu scores for a particular overall value, 
# and then group them as one value for that overall score
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.028146
5,2.130005
